# Import de librerías y preparación de data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
ruta = "/content/drive/My Drive/Maestria/Mineria de datos/Dataset proyecto/"

In [4]:
archivos = glob.glob(os.path.join(ruta, "*.xlsx"))
lista_df = [pd.read_excel(f) for f in archivos]
datos = pd.concat(lista_df, ignore_index=True)

In [5]:
datos.head()

,AÑO,MES,DIASESTAN,SEXO,PPERTENENCIA,EDAD,PERIODOEDA,DEPTORESIDEN,MUNIRESIDEN,CAUFIN,CONDIEGRES,TRATARECIB,DIASESTANCIA
0,2024,1,1.0,2,1,5,2,1,101,A010,1,1,NaN
1,2024,1,1.0,2,1,1,3,1,101,A010,1,1,NaN
2,2024,1,2.0,1,4,3,3,1,101,A010,1,1,NaN
3,2024,1,2.0,2,1,6,3,1,101,A010,1,1,NaN
4,2024,2,3.0,1,9,6,3,1,101,A010,1,1,NaN


In [6]:
datos_nn3 = datos.copy()
datos_nn3 = datos_nn3[
    (datos_nn3["EDAD"].between(0, 99)) &
    (datos_nn3["DIASESTAN"].between(1, 98))
]

# Nos quedamos solo con TRATARECIB 1, 2, 3
datos_nn3 = datos_nn3[datos_nn3["TRATARECIB"].isin([1, 2, 3])]

datos_nn3 = datos_nn3.dropna(subset=["EDAD", "SEXO", "PPERTENENCIA", "DIASESTAN", "TRATARECIB"])

X3 = datos_nn3[["EDAD", "SEXO", "PPERTENENCIA", "DIASESTAN"]].copy()
y3 = datos_nn3["TRATARECIB"].copy()

In [9]:
# Codificar salida como 0,1,2 para usar softmax
clases_trata = sorted(y3.unique())  # [1,2,3]
mapa_trata3 = {clase: idx for idx, clase in enumerate(clases_trata)}
y3_int = y3.map(mapa_trata3).values
y3_cat = to_categorical(y3_int)

# Normalización
scaler3 = StandardScaler()
X3_scaled = scaler3.fit_transform(X3)


In [10]:
# Train / Test RN3
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3_scaled, y3_cat, test_size=0.2, random_state=42
)

model3 = Sequential()
model3.add(Dense(32, activation='relu', input_dim=X3_train.shape[1]))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(y3_cat.shape[1], activation='softmax'))

model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
print("\n=== Entrenando RN3 – Tipo de Tratamiento (TRATARECIB) ===")
history3 = model3.fit(
    X3_train, y3_train,
    epochs=25,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)


=== Entrenando RN3 – Tipo de Tratamiento (TRATARECIB) ===
Epoch 1/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8157 - loss: 0.5211 - val_accuracy: 0.8465 - val_loss: 0.4105
Epoch 2/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8439 - loss: 0.4143 - val_accuracy: 0.8470 - val_loss: 0.4053
Epoch 3/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8443 - loss: 0.4102 - val_accuracy: 0.8478 - val_loss: 0.4048
Epoch 4/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8443 - loss: 0.4107 - val_accuracy: 0.8476 - val_loss: 0.4033
Epoch 5/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8443 - loss: 0.4078 - val_accuracy: 0.8468 - val_loss: 0.4034
Epoch 6/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8433 - loss: 0.4093 - val_accuracy: 0.8477 - val_loss: 0.4026
Epoch 7/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8455 - loss: 0.4056 - val_accuracy: 0.8472 - val_loss: 0.4036
Epoch 8/25
1728/1728 ━━━━━━━━━━

In [13]:
loss3, acc3 = model3.evaluate(X3_test, y3_test, verbose=0)
print(f"Precisión RN3 en test: {acc3:.4f}")
print(f'Loss: {loss3}, Accuracy: {acc3}')
print("Clases (índice → TRATARECIB):", mapa_trata3)

Precisión RN3 en test: 0.8433
Loss: 0.40938061475753784, Accuracy: 0.8432705998420715
Clases (índice → TRATARECIB): {np.int64(1): 0, np.int64(2): 1, np.int64(3): 2}


# Predicciones de escenarios

In [14]:
esc3_A = pd.DataFrame({
    "EDAD": [22],
    "SEXO": [2],
    "PPERTENENCIA": [1],
    "DIASESTAN": [1]
})

esc3_B = pd.DataFrame({
    "EDAD": [50],
    "SEXO": [1],
    "PPERTENENCIA": [2],
    "DIASESTAN": [4]
})

esc3_C = pd.DataFrame({
    "EDAD": [10],
    "SEXO": [2],
    "PPERTENENCIA": [3],
    "DIASESTAN": [2]
})

esc3_D = pd.DataFrame({
    "EDAD": [70],
    "SEXO": [1],
    "PPERTENENCIA": [1],
    "DIASESTAN": [7]
})

In [15]:
def predecir_rn3(escenario_df, nombre):
    esc_scaled = scaler3.transform(escenario_df)
    prob = model3.predict(esc_scaled)[0]
    idx = np.argmax(prob)
    inv_mapa = {v: k for k, v in mapa_trata3.items()}
    clase_trata = inv_mapa[idx]
    print(f"\nEscenario {nombre} – RN3")
    print("Probabilidades por tipo de tratamiento (TRATARECIB):")
    for i, p in enumerate(prob):
        print(f"  {inv_mapa[i]}: {p:.3f}")
    print(f"Clasificación final TRATARECIB: {clase_trata}")

predecir_rn3(esc3_A, "A")
predecir_rn3(esc3_B, "B")
predecir_rn3(esc3_C, "C")
predecir_rn3(esc3_D, "D")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step

Escenario A – RN3
Probabilidades por tipo de tratamiento (TRATARECIB):
  1: 0.401
  2: 0.137
  3: 0.462
Clasificación final TRATARECIB: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

Escenario B – RN3
Probabilidades por tipo de tratamiento (TRATARECIB):
  1: 0.809
  2: 0.191
  3: 0.000
Clasificación final TRATARECIB: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

Escenario C – RN3
Probabilidades por tipo de tratamiento (TRATARECIB):
  1: 0.884
  2: 0.109
  3: 0.007
Clasificación final TRATARECIB: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Escenario D – RN3
Probabilidades por tipo de tratamiento (TRATARECIB):
  1: 0.867
  2: 0.133
  3: 0.000
Clasificación final TRATARECIB: 1
